In [40]:
##ScrapeLangmuirData
##Written by Rebecca Rapf
##June 10, 2021

###Read in csv's of Langmuir trough data and export the useful bits for loading into Igor

##import relevant libraries
import pandas as pd
import numpy as np
import glob
#import matplotlib as mpl
#import matplotlib.pyplot as plt 
import os
#import csv


##Parameters to be changed for each analysis##
FilePath = "/Users/rebeccarafp/Documents/RapfLabPython/LangmuirData/NaClBen/3mMBen" ##currently local only, will change to read from google drive
OutFolder = "Processed"
FileType = 'txt' #pick from "xlsx" or "csv"
Surfactant = "myristic_3mMNaClBen" #name of surfactant and/or conditions
#StockConcSame = "yes" #"yes" or "no" if same concentration was used for each run



StockConc = 1#4.18 #mM 
MM = 284.48 #molecular weight in (g/mol)
VolDeposit_uL = 15 #volume stock solution deposited in 
########

##File-Specific Parameters####
header_length = 43 #skip the first 43 lines in the file
#########

##Constants Library##
AVOGADRO = 6.022*10**23
uL_CONVERSION = 10**6 #number of uL in 1 L
Ang_CONVERSION = 10**10 #number of Angstroms in 1 m
cm_CONVERSION = 10**2 #number of cm in 1 m
cm2_to_Ang2 = Ang_CONVERSION**2/cm_CONVERSION**2
#print(cm2_to_Ang2)
###########

####Calculate number of molecules deposited###
VolDeposit_L = VolDeposit_uL/uL_CONVERSION
AmountDeposit = ((StockConc/MM))*(VolDeposit_L)
NumMolecules = AmountDeposit*AVOGADRO
#print(NumMolecules)

##Set file path to where data is being held locally
os.chdir(FilePath)
print(os.getcwd())


/Users/rebeccarafp/Documents/RapfLabPython/LangmuirData/NaClBen/3mMBen


In [41]:
files = sorted(glob.glob("*"+FileType))
files.sort(key=os.path.getmtime)
print(files)

num_files = len(files)

AllFiles = pd.DataFrame()

for entry in range(num_files):
    name = files[entry]
    outname = name[:-(len(FileType)+1)]
    #print(entry)
    #print(name)
    if FileType == "txt":
        CurrFile = pd.read_csv(files[entry], delimiter = "\t",encoding = 'unicode_escape') 
        CurrFile = CurrFile.rename(columns={CurrFile.columns[1]: "A_cm2"})
        CurrFile = CurrFile.rename(columns={CurrFile.columns[5]: "P_mN_m"})
        A_Ang2 = (CurrFile["A_cm2"]*cm2_to_Ang2)
        MMA=A_Ang2/NumMolecules
        
        IndFile = pd.DataFrame(columns = [outname+"_Acm2",outname+"_P",outname+"_MMA"])
        IndFile[outname+"_Acm2"]=CurrFile["A_cm2"]
        IndFile[outname+"_P"]=CurrFile["P_mN_m"]
        IndFile[outname+"_MMA"]=MMA
        
        AllFiles[outname+"_Acm2"]=CurrFile["A_cm2"]
        AllFiles[outname+"_P"]=CurrFile["P_mN_m"]
        AllFiles[outname+"_MMA"]=MMA
        
        #print(CurrFile)
#     if FileType == "csv":
#         CurrFile = pd.read_csv(files[entry],skiprows = header_length)
    elif FileType == "xlsx":
        #CompFile = files[entry]
        #print(CompFile)
        CurrFile = pd.read_excel(files[entry],skiprows = header_length)
        #print(CurrFile)
        #print("excel")
        CurrFile = CurrFile.rename(columns={CurrFile.columns[3]: "A_cm2"})
        CurrFile = CurrFile.rename(columns={CurrFile.columns[4]: "MMA"})
        MMA = CurrFile["MMA"]
        print(CurrFile)
        
#         #MMACheck = CurrFile["MMA"]
#         #print(CurrFile.iloc[0,4])
        if np.isnan(CurrFile.iloc[0,4]) == True:
            A_Ang2 = (CurrFile["A_cm2"]*cm2_to_Ang2)
            #CurrFile.insert(4, "A_Ang2", A_Ang2)
            MMA=A_Ang2/NumMolecules
            #print(MMA)
            #print("fixing")
        
        
        IndFile = pd.DataFrame(columns = [outname+"_Acm2",outname+"_P",outname+"_MMA"])
        IndFile[outname+"_Acm2"]=CurrFile["A_cm2"]
        IndFile[outname+"_P"]=CurrFile["P1[mN/m]"]
        IndFile[outname+"_MMA"]=CurrFile["MMA"]
        
        AllFiles[outname+"_Acm2"]=CurrFile["A_cm2"]
        AllFiles[outname+"_P"]=CurrFile["P1[mN/m]"]
        AllFiles[outname+"_MMA"]=MMA
        
        IndFile.to_csv("../../"+OutFolder+"/"+outname+"_Processed.csv")
 
AllFiles.to_csv("../../"+OutFolder+"/"+Surfactant+"_AllProcessed.csv")


['3mMnacl_1.txt', '15ulmyristic3mMnacl_1.txt', 'root1_myristic3mMnacl_2.txt', 'root2_myristic3mMnacl_3.txt']
